# Qwen Coder - Simple Working Version

This version focuses on **reliability** - always returns a response.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('✅ Setup done')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('\n✅ Model ready')

In [ ]:
# SIMPLE RELIABLE API - Ignores tools, always returns content
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading

app = Flask(__name__)

@app.route('/v1/models', methods=['GET'])
def models():
    return jsonify({"object": "list", "data": [{"id": "qwen2.5-coder:7b", "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    
    # Clean messages - only keep role and content
    clean_messages = []
    for m in messages:
        role = m.get('role', 'user')
        content = m.get('content', '')
        if role == 'tool':
            role = 'user'
            content = f"Tool result: {content}"
        if content:  # Only add if there's content
            clean_messages.append({'role': role, 'content': str(content)})
    
    if not clean_messages:
        clean_messages = [{'role': 'user', 'content': 'Hello'}]
    
    # Call Ollama - simple, no tools
    content = "I'm ready to help with your coding task!"
    try:
        print(f"Calling Ollama with {len(clean_messages)} messages...")
        r = req.post('http://localhost:11434/api/chat', json={
            'model': 'qwen2.5-coder:7b',
            'messages': clean_messages,
            'stream': False,
            'options': {'num_ctx': 8192, 'temperature': 0.7}
        }, timeout=120)
        
        if r.status_code == 200:
            result = r.json()
            content = result.get('message', {}).get('content', '')
            print(f"Got response: {content[:100]}...")
        else:
            print(f"Ollama error: {r.status_code}")
            content = "I encountered an issue but I'm here to help. Please try again."
            
    except Exception as e:
        print(f"Exception: {e}")
        content = f"I'm having trouble connecting. Please try again."
    
    # GUARANTEE content is never empty
    if not content or not content.strip():
        content = "I understand. How can I help you with your code?"
    
    response = {
        "id": f"chatcmpl-{uuid.uuid4().hex[:12]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": "qwen2.5-coder:7b",
        "choices": [{
            "index": 0,
            "message": {"role": "assistant", "content": content},
            "finish_reason": "stop"
        }],
        "usage": {"prompt_tokens": 50, "completion_tokens": len(content.split()), "total_tokens": 50 + len(content.split())}
    }
    print(f"Returning: {json.dumps(response)[:200]}...")
    return jsonify(response)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"})

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(3)
print('\n✅ Simple API running on port 5000')

In [ ]:
# Quick test
import requests
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Write hello world in Python'}]
})
print("Response:", r.json()['choices'][0]['message']['content'][:300])

In [ ]:
# Start Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''<div style="background:#2ecc71;padding:25px;border-radius:15px">
            <h2 style="color:white">✅ API Ready!</h2>
            <p style="color:white;font-size:20px;font-family:monospace">{url}/v1</p>
            </div>'''))
            break

for line in tunnel.stdout:
    print(line, end='')